In [1]:
import torch

from tqdm import tqdm
from model.continuous_prompt import ContinuousPromptingLLM
from model.text_encoder import TextContinuousPromptModel
from model.projection import BasicProjection
from transformers import AutoTokenizer, AutoModelForCausalLM

from dataset import Dataset
from util import convert_answer

In [2]:
MODE='test'
TASK='cycle_check'
MODEL_NAME = 'bert'
SAVE_DIR=f'/home/bonbak/GraphToken/output/{TASK}'
TASKS_DIR = f'/home/bonbak/GraphToken/task/{TASK}'
DEVICE='cuda:3'

In [3]:
test_dataset = Dataset(f"{TASKS_DIR}/{MODE}.jsonl")
print('\n'.join((test_dataset.data[0]['node_information'], test_dataset.data[0]['edge_information'], test_dataset.data[0]['question'])))

G describes a graph among nodes 0, 1, 2, 3, 4, 5, 6, and 7.
The edges in G are: (0, 1) (0, 5) (0, 6) (2, 6) (4, 7).
You must answer with "Yes" or "No" under the question.
Q: Is there a cycle in this graph?
A: 


In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
).to(DEVICE)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
model.eval()
pred = []
label = []

for input_text, continuous_prompt_input, answer_list in tqdm(test_dataset):
    input_text = '\n'.join((continuous_prompt_input['input_text_list'], input_text))
    input_ids = tokenizer(input_text, return_tensors="pt").to(DEVICE)
    outputs = model.generate(**input_ids, max_new_tokens=4)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    answer = answer.replace(input_text, '')

    pred.append(answer)
    label.append(answer_list)

100%|██████████| 700/700 [01:21<00:00,  8.57it/s]


In [33]:
y_pred, missed = convert_answer(pred)
y_true, _ = convert_answer(label)

In [34]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(accuracy)
print(f1)

0.6585714285714286
0.7941429801894918
